<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/Trending_Assets_Coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install ta
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=ae7df3da5a302e161d703b9177b1ec80d7a2cec3546f896df269a82f0037d762
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl s

In [3]:
import requests
import pandas as pd
import time
from datetime import datetime

# Fetch Trending Assets from CoinGecko
def fetch_trending_assets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 250,
        "page": 1,
        "sparkline": "false"
    }

    all_trending_assets = []

    print("Fetching data...")
    for page in range(1, 21):  # 20 pages * 250 = 5000 assets max
        params["page"] = page
        response = requests.get(url, params=params)
        if response.status_code == 429:
            print("Rate limit reached. Retrying after 60 seconds...")
            time.sleep(60)  # Wait 1 minute for rate limits to reset
            continue
        elif response.status_code != 200:
            print(f"Failed to fetch data from page {page}. Status code: {response.status_code}")
            break

        data = response.json()
        if not data:  # No more data
            break

        # Filter for trending assets
        for asset in data:
            price_change_1d = asset.get("price_change_percentage_24h", 0)
            if abs(price_change_1d or 0) >= 60:
                current_price = asset["current_price"]
                stop_loss = current_price * 0.95
                take_profit = current_price + (current_price - stop_loss) * 2

                all_trending_assets.append({
                    "Name": asset["name"],
                    "Symbol": asset["symbol"],
                    "Price (USD)": current_price,
                    "Market Cap (USD)": asset["market_cap"],
                    "Volume (24h) (USD)": asset["total_volume"],
                    "1D % Change": price_change_1d,
                    "Stop Loss (USD)": stop_loss,
                    "Take Profit (USD)": take_profit
                })

        print(f"Processed page {page}")
        time.sleep(1)  # Avoid rate limiting

    return all_trending_assets

# Save Results to CSV
def save_to_csv(data):
    if data:
        df = pd.DataFrame(data)
        filename = f"trending_assets_{datetime.now().strftime('%Y-%m-%d')}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved trending assets to {filename}")
        return filename
    else:
        print("No trending assets found.")
        return None

# Main Script
if __name__ == "__main__":
    trending_assets = fetch_trending_assets()
    if trending_assets:
        for asset in trending_assets[:5]:  # Print first 5 assets as a preview
            print(asset)
        file = save_to_csv(trending_assets)

        # Download file in Colab
        if file:
            from google.colab import files
            files.download(file)
    else:
        print("No assets with ±60% 1D changes found.")


Fetching data...
Processed page 1
Processed page 2
Processed page 3
Processed page 4
Processed page 5
Rate limit reached. Retrying after 60 seconds...
Processed page 7
Processed page 8
Processed page 9
Processed page 10
Processed page 11
Processed page 12
Rate limit reached. Retrying after 60 seconds...
Processed page 14
Rate limit reached. Retrying after 60 seconds...
Processed page 16
Processed page 17
Processed page 18
Processed page 19
Processed page 20
{'Name': 'Official Trump', 'Symbol': 'trump', 'Price (USD)': 70.2, 'Market Cap (USD)': 14116888546, 'Volume (24h) (USD)': 27027259376, '1D % Change': 212.89646, 'Stop Loss (USD)': 66.69, 'Take Profit (USD)': 77.22000000000001}
{'Name': 'Dogelon Mars', 'Symbol': 'elon', 'Price (USD)': 3.19568e-07, 'Market Cap (USD)': 175747472, 'Volume (24h) (USD)': 114034637, '1D % Change': 73.15456, 'Stop Loss (USD)': 3.0358959999999996e-07, 'Take Profit (USD)': 3.5152480000000006e-07}
{'Name': 'MAGA', 'Symbol': 'trump', 'Price (USD)': 2.94, 'Marke

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>